<a href="https://colab.research.google.com/github/gabrieladamasceno/Attacks_5G/blob/main/KNN_Zero_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Acessar dataset
drive.mount('/content/gdrive')

# Carregar o dataset
encoded = pd.read_csv("/content/gdrive/MyDrive/Datasets/Attacks/Encoded/Encoded.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt


In [88]:
# Usando shape
num_amostras = encoded.shape[0]
print(f"Número de amostras: {num_amostras}")

# Usando len()
num_amostras_len = len(encoded)
print(f"Número de amostras (usando len): {num_amostras_len}")

Número de amostras: 1215890
Número de amostras (usando len): 1215890


In [89]:
# Exibir os nomes de todas as colunas
colunas = encoded.columns.tolist()  # Converte o Index em uma lista
print("Nomes das colunas:")
for coluna in colunas:
    print(coluna)

Nomes das colunas:
Unnamed: 0
Seq
Dur
RunTime
Mean
Sum
Min
Max
sTos
dTos
sTtl
dTtl
sHops
dHops
TotPkts
SrcPkts
DstPkts
TotBytes
SrcBytes
DstBytes
Offset
sMeanPktSz
dMeanPktSz
Load
SrcLoad
DstLoad
Loss
SrcLoss
DstLoss
pLoss
SrcGap
DstGap
Rate
SrcRate
DstRate
SrcWin
DstWin
sVid
dVid
SrcTCPBase
DstTCPBase
TcpRtt
SynAck
AckDat
Label
Attack Type
Attack Tool
 *        
 *    V   
 *    f   
 e        
 e    f   
 e &      
 e *      
 e d      
 e g      
 e i      
 e r      
 e s      
 eU       
e        
arp
icmp
ipv6-icmp
llc
lldp
sctp
tcp
udp
ACC
CON
ECO
FIN
INT
NRS
REQ
RSP
RST
TST
URP
Shutdown
Start
Status
39
4
52
54
af11
af12
af41
cs0
cs4
cs6
cs7
ef
nan


In [90]:
# Selecionar apenas colunas numéricas
numeric_cols = encoded.select_dtypes(include=['number']).columns

# Aplicar a suavização apenas nas colunas numéricas
for col in numeric_cols:
    encoded[col] = encoded[col].fillna(encoded[col].mean())

In [91]:
# columns with null values

columns_null = []
columns = encoded.columns
for column in columns:
    c = encoded[column].isnull().sum()
    if c != 0:
        print(column, 'has {} null values'.format(c))
        columns_null.append(column)

Features
* Seq: Sequência de pacotes.
* Offset: Deslocamento em bytes.
* sTtl: Time to Live da fonte.
* tcp: Indica se o protocolo TCP está sendo usado.
* AckDat: Dados de reconhecimento.
* RST: Sinalizador de reinício TCP.
* INT: Interrupções no fluxo de dados.
* TcpRtt: Tempo de ida e volta do TCP.
* icmp: Indica se o protocolo ICMP está presente.
* sMeanPktSz: Tamanho médio do pacote da fonte.
* FIN: Sinalizador de finalização de conexão.
* sHops: Número de saltos até o destino.
* Mean: Valor médio de um conjunto de dados.
* dTtl: Time to Live do destino.
* SrcBytes: Bytes enviados da fonte.
* TotBytes: Total de bytes.
* dMeanPktSz: Tamanho médio do pacote do destino.
* Status: Estado da conexão.
* cs0: Parâmetro de controle.
* SrcWin: Janela de origem TCP.
* sTos: Tipo de serviço.
* CON: Conexões.
* REQ: Requisições.
* Loss: Taxa de perda de pacotes.

In [92]:
import pandas as pd

# Selecionar apenas as colunas 'Label' e 'Attack Type'
icmp_flood_samples = encoded[['Label', 'icmp', 'Attack Type']]

icmp_flood_samples

,Label,icmp,Attack Type
0,Benign,1.0,Benign
1,Benign,1.0,Benign
2,Benign,0.0,Benign
3,Benign,0.0,Benign
4,Benign,0.0,Benign
...,...,...,...
1215885,Benign,0.0,Benign
1215886,Benign,0.0,Benign
1215887,Benign,0.0,Benign
1215888,Benign,0.0,Benign


In [93]:
import psutil
import time
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Função para monitorar recursos
def monitor_resource_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    cpu_percent = psutil.cpu_percent(interval=0.1)
    memory_usage_mb = memory_info.rss / (1024 * 1024)
    return cpu_percent, memory_usage_mb

# Callback personalizado para monitorar CPU e memória
class ResourceMonitor(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        # Antes da época começar
        self.start_time = time.time()
        self.start_cpu, self.start_memory = monitor_resource_usage()
        print(f"\n[Início da Época {epoch + 1}]")
        print(f"CPU inicial: {self.start_cpu:.2f}%, Memória inicial: {self.start_memory:.2f} MB")

    def on_epoch_end(self, epoch, logs=None):
        # Após a época terminar
        end_time = time.time()
        end_cpu, end_memory = monitor_resource_usage()
        elapsed_time = end_time - self.start_time
        memory_diff = end_memory - self.start_memory

        print(f"[Fim da Época {epoch + 1}]")
        print(f"CPU final: {end_cpu:.2f}%, Memória consumida: {memory_diff:.2f} MB, Tempo gasto: {elapsed_time:.2f} segundos")

# Instanciar o callback
resource_monitor = ResourceMonitor()

# Parâmetros para early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Configuração das features e rótulos
features = [
    'Seq', 'Offset', 'sTtl', 'tcp', 'AckDat', 'RST', 'INT', 'TcpRtt', 'icmp',
    'sMeanPktSz', 'FIN', 'sHops', 'Mean', 'dTtl', 'SrcBytes', 'TotBytes',
    'dMeanPktSz', 'Status', 'cs0', 'SrcWin', 'sTos', 'CON', 'REQ', 'Loss', 'Attack Type'
]

X = encoded[features]
y = encoded['Label']  # Coluna de rótulos

# Divida os dados normalmente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifique as amostras ICMPFlood no conjunto de treino
train_zero_day_indices = X_train[X_train['Attack Type'] == 'ICMPFlood'].index

# Remova essas amostras de ICMPFlood do conjunto de treino
X_train_clean = X_train.drop(train_zero_day_indices)
y_train_clean = y_train.drop(train_zero_day_indices)

# Identifique as amostras ICMPFlood no conjunto de teste
test_zero_day_indices = X_test[X_test['Attack Type'] == 'ICMPFlood'].index

# Mantenha as amostras de ICMPFlood no conjunto de teste
X_test_zero_day = X_test.loc[test_zero_day_indices]
y_test_zero_day = y_test.loc[test_zero_day_indices]

# Verificar a distribuição das classes no conjunto de teste
print(f"Zero-day presente no conjunto de teste: {any(test_zero_day_indices.isin(X_test_zero_day.index))} \n")

# Verificar as distribuições
print("Distribuição do conjunto de treino binário:", pd.Series(y_train_clean).value_counts())
print("Distribuição do conjunto de teste binário:", pd.Series(y_test_zero_day).value_counts())

# Remover a coluna 'Attack Type' das features de treino e teste
X_train_clean = X_train_clean.drop(columns=['Attack Type'])
X_test = X_test.drop(columns=['Attack Type'])
X_test_zero_day = X_test_zero_day.drop(columns=['Attack Type'])

Zero-day presente no conjunto de teste: True 

Distribuição do conjunto de treino binário: Label
Malicious    589852
Benign       381943
Name: count, dtype: int64
Distribuição do conjunto de teste binário: Label
Malicious    238
Name: count, dtype: int64


In [95]:
y_test

,Label
873982,Malicious
617114,Malicious
874834,Malicious
1065744,Benign
876351,Malicious
...,...
818966,Malicious
1166938,Benign
669203,Benign
948209,Malicious


In [96]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd

# Configurar o SMOTE
smote = SMOTE(random_state=42)

# Aplicar SMOTE no conjunto de treinamento
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_clean, y_train_clean)

# Verificar distribuição das classes após o SMOTE
print(f"Distribuição após SMOTE:\n{pd.Series(y_train_balanced).value_counts()}")

# Normalizar os dados após o SMOTE
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)

# Normalizar o conjunto de teste com o mesmo scaler ajustado no treino
X_test_combined = scaler.transform(X_test)
y_test_combined = y_test

Distribuição após SMOTE:
Label
Malicious    589852
Benign       589852
Name: count, dtype: int64


In [97]:
# Verificar a dimensão do treino sem balancear
#print(X_train_non_zero_day.shape)
#print(y_train_non_zero_day.shape)
#print(f"Distribuição sem balancear:\n{pd.Series(y_train_non_zero_day).value_counts()}")

# Verificar a dimensão do teste
print(X_test_combined.shape)
print(y_test_combined.shape)
print(f"Distribuição Teste:\n{pd.Series(y_test_combined).value_counts()}")

(243178, 24)
(243178,)
Distribuição Teste:
Label
Malicious    147384
Benign        95794
Name: count, dtype: int64


In [98]:
from sklearn.preprocessing import LabelEncoder

# Criar o codificador
label_encoder = LabelEncoder()

# Ajustar o codificador aos rótulos de treinamento e teste
y_train_balanced = label_encoder.fit_transform(y_train_balanced)
y_test_combined = label_encoder.transform(y_test_combined)

# Verificar os tipos após a transformação
print(y_train_balanced.dtype)
print(y_test_combined.dtype)

int64
int64


In [99]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Configuração do modelo KNN
k = 10  # Número de vizinhos
knn_model = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric='euclidean')

# Monitorar antes e depois do treinamento
print("[Monitoramento de Recursos durante o ajuste do modelo KNN]")
start_cpu, start_memory = monitor_resource_usage()
start_time = time.time()

# Treinar o modelo KNN
knn_model.fit(X_train_balanced, y_train_balanced)

end_cpu, end_memory = monitor_resource_usage()
end_time = time.time()

# Exibir informações de uso de recursos
print(f"CPU inicial: {start_cpu:.2f}%, Memória inicial: {start_memory:.2f} MB")
print(f"CPU final: {end_cpu:.2f}%, Memória final: {end_memory:.2f} MB")
print(f"Memória consumida: {end_memory - start_memory:.2f} MB")
print(f"Tempo total: {end_time - start_time:.2f} segundos")

[Monitoramento de Recursos durante o ajuste do modelo KNN]
CPU inicial: 1.70%, Memória inicial: 6326.14 MB
CPU final: 1.80%, Memória final: 6542.04 MB
Memória consumida: 215.90 MB
Tempo total: 0.22 segundos


In [100]:
# Monitorar durante a predição
print("[Monitoramento de Recursos durante a predição]")
start_cpu, start_memory = monitor_resource_usage()
start_time = time.time()

y_pred = knn_model.predict(X_test_combined)

end_cpu, end_memory = monitor_resource_usage()
end_time = time.time()

# Exibir informações de uso de recursos
print(f"CPU inicial: {start_cpu:.2f}%, Memória inicial: {start_memory:.2f} MB")
print(f"CPU final: {end_cpu:.2f}%, Memória final: {end_memory:.2f} MB")
print(f"Memória consumida: {end_memory - start_memory:.2f} MB")
print(f"Tempo total: {end_time - start_time:.2f} segundos")

[Monitoramento de Recursos durante a predição]
CPU inicial: 2.00%, Memória inicial: 6542.04 MB
CPU final: 0.20%, Memória final: 6542.04 MB
Memória consumida: 0.00 MB
Tempo total: 47.25 segundos


In [109]:
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score

# Definindo o classificador
knn = KNeighborsClassifier(n_neighbors=5)

# Suponha que X_train_clean e y_train_clean sejam seus dados de treinamento
# Calculando múltiplas métricas com 5-fold cross-validation
scoring = ['accuracy', 'precision', 'recall', 'f1']
results = cross_validate(knn, X_train_clean, y_train_clean, cv=5, scoring=scoring)

# Exibindo os resultados para cada fold
print("Acurácia para cada fold:", results['test_accuracy'])
print("Precisão para cada fold:", results['test_precision'])
print("Recall para cada fold:", results['test_recall'])
print("F1-score para cada fold:", results['test_f1'])

# Média das métricas
print("Acurácia média:", results['test_accuracy'].mean())
print("Precisão média:", results['test_precision'].mean())
print("Recall médio:", results['test_recall'].mean())
print("F1-score médio:", results['test_f1'].mean())

# Agora, realizando a previsão e convertendo para 'Benign'/'Malicious'
y_pred = knn.predict(X_test_clean)  # Suponha que X_test_clean seja o conjunto de teste
y_pred = ['Benign' if label == 0 else 'Malicious' for label in y_pred]

# Calculando a precisão usando a conversão correta para 'Malicious'
precision = precision_score(y_test, y_pred, pos_label='Malicious')  # ou 'Benign', dependendo do caso
print("Precisão no conjunto de teste:", precision)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py", line 2247, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
                 ^^^^^^^^

Acurácia para cada fold: [0.98763628 0.98753852 0.98776491 0.9877752  0.98769802]
Precisão para cada fold: [nan nan nan nan nan]
Recall para cada fold: [nan nan nan nan nan]
F1-score para cada fold: [nan nan nan nan nan]
Acurácia média: 0.9876825873769673
Precisão média: nan
Recall médio: nan
F1-score médio: nan


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py", line 2247, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
                 ^^^^^^^^

NameError: name 'X_test_clean' is not defined

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

# Suponha que X_train e y_train sejam seus dados de treinamento
knn = KNeighborsClassifier(n_neighbors=5)

# Calculando a AUC com 5-fold cross-validation
auc_scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='roc_auc')

print("AUC para cada fold:", auc_scores)
print("AUC média:", auc_scores.mean())

In [ ]:
# Avaliar o desempenho do modelo
print("\nRelatório de Classificação:")
print(classification_report(y_test_combined, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_test_combined, y_pred)
print("\nMatriz de Confusão:")
print(cm)

# Opcional: Visualizar a matriz de confusão
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_).plot(cmap='Blues')
plt.show()

In [ ]:
X_test_zero_day_no_names = X_test_zero_day.values

y_pred_zero_day = knn_model.predict(X_test_zero_day_no_names)

In [ ]:
# Verifique se as dimensões de X_test_zero_day e y_test_zero_day são consistentes
print(f"Tamanho de X_test_zero_day: {X_test_zero_day.shape}")
print(f"Tamanho de y_test_zero_day: {y_test_zero_day.shape}")


In [ ]:
# Prever com o modelo para as amostras zero-day
y_pred_zero_day = knn_model.predict(X_test_zero_day)

# Verifique as previsões
print(f"Previsões para as amostras zero-day: {y_pred_zero_day}")

# Verifique se as previsões estão corretas (se a classe de 'maligno' é representada como 1)
accuracy_zero_day = (y_pred_zero_day == 1).mean()  # Supondo que '1' é a classe de maligno
print(f"Acurácia para zero-day: {accuracy_zero_day * 100:.2f}%")


In [ ]:
# Comparando previsões com os rótulos reais
correct_classifications = (y_pred_zero_day == y_test_zero_day).sum()
total_zero_day_samples = len(y_test_zero_day)

print(f"Amostras zero-day classificadas corretamente: {correct_classifications}/{total_zero_day_samples}")
print(f"Taxa de acerto para as amostras zero-day: {correct_classifications / total_zero_day_samples * 100:.2f}%")